In [ ]:
# ==============================================================================
# PROJECT: Financial Market Data Pipeline & Analytics
# AUTHOR: Fernando G. Sloboda
# DESCRIPTION: Automated ETL pipeline for financial market data extraction,
#              cleaning, time-series analysis, and performance visualization.
# ==============================================================================

# --- Imports ---
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import os
import warnings

# Suppress minor warnings for clean output
warnings.filterwarnings('ignore')

# --- Global Configurations ---
ASSETS = {
    "CL=F": "Crude Oil (USD)",
    "GC=F": "Gold (USD)",
    "ZC=F": "Corn (USD)",
    "BRL=X": "USD/BRL"
}
DAYS_TO_FETCH = 30
OUTPUT_DIR = './data/'

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*70)
print("📊 FINANCIAL MARKET DATA PIPELINE - QUANTITATIVE ANALYSIS")
print("="*70)

# ==============================================================================
# PHASE 1: DATA INGESTION (API EXTRACTION)
# ==============================================================================
print("\n[INFO] PHASE 1: Data Ingestion Initiated")

end_date = datetime.today()
start_date = end_date - timedelta(days=DAYS_TO_FETCH)
tickers = list(ASSETS.keys())

print(f"[+] Target Assets: {', '.join(ASSETS.values())}")
print(f"[+] Timeframe: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

try:
    # Fetch data via Yahoo Finance API
    df_market = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=False)
    
    if df_market.empty:
        raise ValueError("API returned an empty dataset. Check tickers or network connection.")
    
    print(f"[+] API Connection Successful. Retrieved {len(df_market)} rows.")
    
    # Extract 'Close' prices and rename columns
    close_prices = pd.DataFrame()
    for ticker in tickers:
        if ticker in df_market.columns.levels[0]:
            close_prices[ticker] = df_market[ticker]['Close']
            
    close_prices.columns = ASSETS.values()
    
    print("[+] Raw Data Preview (Last 3 days):")
    print(close_prices.tail(3))

except Exception as e:
    print(f"[ERROR] Pipeline Failed during Data Ingestion: {e}")
    exit()

# ==============================================================================
# PHASE 2: DATA TRANSFORMATION & METRICS
# ==============================================================================
print("\n[INFO] PHASE 2: Data Cleaning & Metric Calculation Initiated")

# 1. Forward Fill for missing data (Holidays/Weekends) and drop initial NaNs
print("[+] Handling missing time-series data (Forward Fill approach)...")
clean_prices = close_prices.ffill().dropna()

# 2. Calculate Daily Returns
print("[+] Calculating Daily Returns (%) and Volatility...")
daily_returns = clean_prices.pct_change().dropna()

# 3. Calculate Key Performance Indicators (KPIs)
total_return = ((clean_prices.iloc[-1] / clean_prices.iloc[0]) - 1) * 100
volatility = daily_returns.std() * np.sqrt(252) * 100 # Annualized Volatility estimation

print("\n--- MARKET SUMMARY (LAST 30 DAYS) ---")
for asset in ASSETS.values():
    t_ret = total_return[asset]
    vol = volatility[asset]
    print(f" > {asset.ljust(18)} | Total Return: {t_ret:>6.2f}% | Ann. Volatility: {vol:>6.2f}%")

# ==============================================================================
# PHASE 3: VISUALIZATION & DATA EXPORT
# ==============================================================================
print("\n[INFO] PHASE 3: Data Visualization & Export Initiated")



# 1. Base 100 Normalization for comparative performance
normalized_prices = (clean_prices / clean_prices.iloc[0]) * 100

# 2. Plotting Configuration
plt.figure(figsize=(14, 7))
plt.style.use('default') # Ensures clean background
colors = ['#2ca02c', '#ff7f0e', '#8c564b', '#1f77b4'] # Corporate color palette

for i, col in enumerate(normalized_prices.columns):
    plt.plot(normalized_prices.index, normalized_prices[col], label=col, linewidth=2.5, color=colors[i])

# Chart Formatting
plt.title('Normalized Asset Performance (Base 100) - Last 30 Days', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Date', fontsize=12, fontweight='bold')
plt.ylabel('Normalized Value (Start = 100)', fontsize=12, fontweight='bold')

# Customize grid and axes
plt.grid(True, which='major', linestyle='--', alpha=0.6)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Format X-axis dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.gcf().autofmt_xdate()

plt.legend(loc='best', frameon=True, shadow=True, fontsize=11)
plt.tight_layout()

# 3. Export Assets
chart_path = os.path.join(OUTPUT_DIR, 'asset_performance_chart.png')
prices_path = os.path.join(OUTPUT_DIR, 'cleaned_market_prices.csv')
returns_path = os.path.join(OUTPUT_DIR, 'daily_returns.csv')

plt.savefig(chart_path, dpi=300, bbox_inches='tight')
clean_prices.to_csv(prices_path)
daily_returns.to_csv(returns_path)

print(f"[+] Performance chart successfully exported to: {chart_path}")
print(f"[+] Dataframes successfully exported to: {OUTPUT_DIR}")

plt.show()

print("\n" + "="*70)
print("✅ PIPELINE EXECUTION COMPLETED SUCCESSFULLY")
print("="*70)

📊 FINANCIAL MARKET DATA PIPELINE - QUANTITATIVE ANALYSIS

[INFO] PHASE 1: Data Ingestion Initiated
[+] Target Assets: Crude Oil (USD), Gold (USD), Corn (USD), USD/BRL
[+] Timeframe: 2026-01-22 to 2026-02-21
[+] API Connection Successful. Retrieved 22 rows.
[+] Raw Data Preview (Last 3 days):
            Crude Oil (USD)   Gold (USD)  Corn (USD)  USD/BRL
Date                                                         
2026-02-18        65.190002  4986.500000      427.00   5.2205
2026-02-19        66.430000  4975.899902      425.75   5.2344
2026-02-20        66.389999  5059.299805      427.50   5.2088

[INFO] PHASE 2: Data Cleaning & Metric Calculation Initiated
[+] Handling missing time-series data (Forward Fill approach)...
[+] Calculating Daily Returns (%) and Volatility...

--- MARKET SUMMARY (LAST 30 DAYS) ---
 > Crude Oil (USD)    | Total Return:  11.84% | Ann. Volatility:  36.14%
 > Gold (USD)         | Total Return:   3.07% | Ann. Volatility:  54.26%
 > Corn (USD)         | Total Ret